In [ ]:
# python 3.11
# please run in the console:
# conda create -n bw25 -c cmutel brightway25
# pip install ipykernel
%pip install numpy pandas plotly

In [ ]:
%pip install bw2io==0.9.dev11

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#import database_explorer as dbex

In [5]:
import bw2analyzer as ba
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import matrix_utils as mu
import bw_processing as bp

first_setup = False
NAME = "ei38-cutoff-25"

if first_setup:
    if NAME in bd.projects:
        bd.projects.delete_project(NAME, True)

bd.projects.set_current(NAME)

if first_setup:
    bi.bw2setup()

In [ ]:
if first_setup:
    ei33 = bi.SingleOutputEcospold2Importer('/home/hurtiol/ALFA/datasets/', 'ecoInvent 3.8')
    ei33.apply_strategies()
    ei33.statistics()


In [4]:
if first_setup:
    ei33.write_database(db_name='ecoInvent 3.8')

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 02/09/2023 10:55:56
  Finished: 02/09/2023 10:59:49
  Total time elapsed: 00:03:53
  CPU %: 62.00
  Memory %: 0.15
Created database: ecoInvent 3.8


In [6]:
act = bd.Database("ecoInvent 3.8").search("biomethane production, high pressure from synthetic gas, wood, fluidised technology")[0]
act

'biomethane production, high pressure from synthetic gas, wood, fluidised technology' (cubic meter, CH, None)

In [7]:
#methods = [x for x in bd.methods if 'IPCC' in x[0]]
method = ('IPCC 2013', 'climate change', 'GWP 100a')

In [8]:
lca = bc.LCA({act: 1}, method = method)
lca.lci()
lca.lcia()
lca.score

0.38520281033490816

In [21]:
# from Romain to name the activities
id_to_key = {v:k for k, v in lca.activity_dict.items()}
labels = {id: bd.get_activity(id_to_key[id])["name"] for id in list(d["nodes"].keys())[1:]}
labels

{8136: 'biomethane production, high pressure from synthetic gas, wood, fluidised technology',
 4040: 'market for wood chips, dry, measured as dry mass',
 5179: 'market for zinc',
 5246: 'market for dolomite',
 9804: 'market for waste mineral oil',
 9946: 'market for zeolite, powder',
 10458: 'market for silica sand',
 13168: 'market for sodium hydroxide, without water, in 50% solution state',
 13946: 'esterification of rape oil',
 14409: 'market for electricity, medium voltage',
 15337: 'market for charcoal',
 16404: 'market for wood chips, wet, measured as dry mass',
 8988: 'wood chips production, softwood, at sawmill',
 12886: 'hardwood forestry, mixed species, sustainable forest management',
 15009: 'softwood forestry, mixed species, sustainable forest management',
 18854: 'market for transport, freight, lorry, unspecified',
 11250: 'transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspecified',
 11726: 'transport, freight, lorry, all size

In [35]:
import warnings
from heapq import heappop, heappush
import itertools
from functools import lru_cache

import numpy as np
from scipy import sparse

from bw2calc import spsolve, LCA

class JRCAssumedDiagonalGraphTraversal:
    """
    Traverse a supply chain, following paths of greatest impact.
    This implementation uses a queue of datasets to assess. As the supply chain is traversed, datasets inputs are added to a list sorted by LCA score. Each activity in the sorted list is assessed, and added to the supply chain graph, as long as its impact is above a certain threshold, and the maximum number of calculations has not been exceeded.
    Because the next dataset assessed is chosen by its impact, not its position in the graph, this is neither a breadth-first nor a depth-first search, but rather "importance-first".
    This class is written in a functional style - no variables are stored in *self*, only methods.
    Should be used by calling the ``calculate`` method.
    .. warning:: Graph traversal with multioutput processes only works when other inputs are substituted (see `Multioutput processes in LCA <http://chris.mutel.org/multioutput.html>`__ for a description of multiputput process math in LCA).
    """

    def calculate(self, lca, cutoff=0.005, max_calc=1e5, skip_coproducts=False):
        """
        Traverse the supply chain graph.
        Args:
            * *lca* (dict): An instance of ``bw2calc.lca.LCA``.
            * *cutoff* (float, default=0.005): Cutoff criteria to stop LCA calculations. Relative score of total, i.e. 0.005 will cutoff if a dataset has a score less than 0.5 percent of the total.
            * *max_calc* (int, default=10000): Maximum number of LCA calculations to perform.
        Returns:
            Dictionary of nodes, edges, and number of LCA calculations.
        """
        if not hasattr(lca, "supply_array"):
            lca.lci()
        if not hasattr(lca, "characterized_inventory"):
            lca.lcia()

        supply = lca.supply_array.copy()
        score = lca.score

        if score == 0:
            raise ValueError("Zero total LCA score makes traversal impossible")

        # Create matrix of LCIA CFs times biosphere flows, as these don't
        # change. This is also the unit score of each activity.
        characterized_biosphere = np.array(
            (lca.characterization_matrix * lca.biosphere_matrix).sum(axis=0)
        ).ravel()
        characterized_biosphere_neg = characterized_biosphere.copy()
        characterized_biosphere_neg[characterized_biosphere_neg > 0] = 0
        characterized_biosphere_pos = characterized_biosphere.copy()
        characterized_biosphere_pos[characterized_biosphere_pos < 0] = 0

        heap, nodes, edges = self.initialize_heap(lca, supply, characterized_biosphere, characterized_biosphere_neg, characterized_biosphere_pos)
        nodes, edges, counter = self.traverse(
            heap,
            nodes,
            edges,
            0,
            max_calc,
            cutoff,
            score,
            supply,
            characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos,
            lca,
            skip_coproducts,
        )

        return {
            "nodes": nodes,
            "edges": edges,
            "counter": counter,
        }

    def initialize_heap(self, lca, supply, characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos):
        """
        Create a `priority queue <http://docs.python.org/2/library/heapq.html>`_ or ``heap`` to store inventory datasets, sorted by LCA score.
        Populates the heap with each activity in ``demand``. Initial nodes are the *functional unit*, i.e. the complete demand, and each activity in the *functional unit*. Initial edges are inputs from each activity into the *functional unit*.
        The *functional unit* is an abstract dataset (as it doesn't exist in the matrix), and is assigned the index ``-1``.
        """
        heap, edges = [], []
        nodes = {-1: {"amount": 1, "cum": lca.score, "ind": 1e-6 * lca.score}}
        for index, amount in enumerate(lca.demand_array):
            if amount == 0:
                continue
            cum_score, cum_score_neg, cum_score_pos = self.cumulative_score(
                index, supply, characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos, lca
            )
            heappush(heap, (abs(1 / cum_score), index))
            nodes[index] = {
                "amount": float(supply[index]),
                "cum": cum_score,
                "cum_neg": cum_score_neg,
                "cum_pos": cum_score_pos,
                "ind": self.unit_score(index, supply, characterized_biosphere),
            }
            edges.append(
                {
                    "to": -1,
                    "from": index,
                    "amount": amount,
                    "exc_amount": amount,
                    "impact": cum_score * amount / float(supply[index]),
                    "impact_neg": cum_score_neg * amount / float(supply[index]),
                    "impact_pos": cum_score_pos * amount / float(supply[index])
                }
            )
        return heap, nodes, edges

    def cumulative_score(self, index, supply, characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos, lca):
        """Compute cumulative LCA score for a given activity"""
        demand = np.zeros((supply.shape[0],))
        demand[index] = (
            supply[index]
            *
            # Normalize by the production amount
            lca.technosphere_matrix[index, index]
        )
        solved_tech = spsolve(lca.technosphere_matrix, demand)
        return (float(
            (characterized_biosphere * solved_tech).sum()
        ),
        float(
            (characterized_biosphere_neg * solved_tech).sum()
        ),
        float(
            (characterized_biosphere_pos * solved_tech).sum()
        ))

    def unit_score(self, index, supply, characterized_biosphere):
        """Compute the LCA impact caused by the direct emissions and resource consumption of a given activity"""
        return float(characterized_biosphere[index] * supply[index])

    def traverse(
        self,
        heap,
        nodes,
        edges,
        counter,
        max_calc,
        cutoff,
        total_score,
        supply,
        characterized_biosphere,
        characterized_biosphere_neg,
        characterized_biosphere_pos,
        lca,
        skip_coproducts,
    ):
        """
        Build a directed graph by traversing the supply chain.
        Node ids are actually technosphere row/col indices, which makes lookup easier.
        Returns:
            (nodes, edges, number of calculations)
        """
        # static_databases = {name for name in databases if databases[name].get("static")}
        # reverse = lca.dicts.activity.reversed

        while heap:
            if counter >= max_calc:
                warnings.warn("Stopping traversal due to calculation count.")
                break
            parent_index = heappop(heap)[1]
            # Skip links from static databases
            # if static_databases and reverse[parent_index][0] in static_databases:
            #     continue

            # Assume that this activity produces its reference product
            scale_value = lca.technosphere_matrix[parent_index, parent_index]
            if scale_value == 0:
                raise ValueError(
                    "Can't rescale activities that produce zero reference product"
                )
            col = lca.technosphere_matrix[:, parent_index].tocoo()
            # Multiply by -1 because technosphere values are negative
            # (consumption of inputs) and rescale
            children = [
                (int(col.row[i]), float(-1 * col.data[i] / scale_value))
                for i in range(col.row.shape[0])
            ]
            for activity, amount in children:
                # Skip values on technosphere diagonal
                if activity == parent_index:
                    continue
                # Skip negative coproducts
                if skip_coproducts and amount <= 0:
                    continue
                counter += 1
                cumulative_score, cum_score_neg, cum_score_pos = self.cumulative_score(
                    activity, supply, characterized_biosphere, characterized_biosphere_neg, characterized_biosphere_pos, lca
                )
                if abs(cumulative_score) < abs(total_score * cutoff):
                    continue

                # flow between activity and parent (Multiply by -1 because technosphere values are negative)
                flow = (
                    -1.0
                    * lca.technosphere_matrix[activity, parent_index]
                    * supply[parent_index]
                )
                total_activity_output = (
                    lca.technosphere_matrix[activity, activity] * supply[activity]
                )

                # Edge format is (to, from, mass amount, cumulative impact)
                edges.append(
                    {
                        "to": parent_index,
                        "from": activity,
                        # Amount of this link * amount of parent demanding link
                        "amount": flow,
                        # Raw exchange value
                        "exc_amount": amount,
                        # Impact related to this flow
                        "impact": flow / total_activity_output * cumulative_score,
                        "impact_neg": flow / total_activity_output * cum_score_neg,
                        "impact_pos": flow / total_activity_output * cum_score_pos
                    }
                )
                # Want multiple incoming edges, but don't add existing node
                if activity in nodes:
                    continue
                nodes[activity] = {
                    # Total amount of this flow supplied
                    "amount": total_activity_output,
                    # Cumulative score from all flows of this activity
                    "cum": cumulative_score,
                    "cum_neg": cum_score_neg,
                    "cum_pos": cum_score_pos,
                    # Individual score attributable to environmental flows
                    # coming directory from or to this activity
                    "ind": self.unit_score(activity, supply, characterized_biosphere),
                }
                heappush(heap, (abs(1 / cumulative_score), activity))

        return nodes, edges, counter


In [36]:
trav = JRCAssumedDiagonalGraphTraversal().calculate(lca)

In [38]:
trav['edges']

[{'to': -1,
  'from': 8136,
  'amount': 1.0,
  'exc_amount': 1.0,
  'impact': 0.38520281033490805,
  'impact_neg': -0.004198531925797447,
  'impact_pos': 0.3894013422607055},
 {'to': 8136,
  'from': 4040,
  'amount': 0.814000010490703,
  'exc_amount': 0.8140000104904175,
  'impact': 0.05148407252302468,
  'impact_neg': -0.0037967297284041976,
  'impact_pos': 0.05528080225142888},
 {'to': 8136,
  'from': 5179,
  'amount': 0.001869400031865299,
  'exc_amount': 0.001869400031864643,
  'impact': 0.005073684967404069,
  'impact_neg': -8.534070839040309e-07,
  'impact_pos': 0.0050745383744879745},
 {'to': 8136,
  'from': 5246,
  'amount': 0.03959999978543717,
  'exc_amount': 0.03959999978542328,
  'impact': 0.0027718753849170484,
  'impact_neg': -2.6693003403092034e-07,
  'impact_pos': 0.002772142314951079},
 {'to': 8136,
  'from': 9804,
  'amount': -0.01462200004607952,
  'exc_amount': -0.01462200004607439,
  'impact': 0.03637634803738445,
  'impact_neg': -4.585507161357012e-08,
  'impact_p

In [ ]:
trav.edges